In [9]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import (
                                                        AzureSearch,
                                                        AzureSearchVectorStoreRetriever
                                                        )
from ragas.llms import LangchainLLM
from ragas import evaluate
from ragas.metrics.critique import harmfulness
from datasets import load_dataset
import PyPDF2
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter

In [2]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["azure_endpoint"] = "https://ai-dev-openai-fts.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "7439c61b0745446dbbb029521f498265"

In [3]:
azure_model = AzureChatOpenAI(
    openai_api_key = "7439c61b0745446dbbb029521f498265",
    azure_endpoint = "https://ai-dev-openai-fts.openai.azure.com/",
    openai_api_version = "2023-07-01-preview",
    openai_organization = "azure",
    temperature=0,    
    deployment_name="gpt35")

In [4]:
azure_embeddings = AzureOpenAIEmbeddings(
            openai_api_key = "7439c61b0745446dbbb029521f498265",
            azure_endpoint = "https://ai-dev-openai-fts.openai.azure.com/",
            openai_api_type = "azure",
            openai_api_version = "2023-07-01-preview",
            deployment="text-embedding-ada-002", 
            chunk_size=1
                                )

In [6]:
def convert_pdf_to_txt(pdf_path, txt_path):
    try:
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ''
            for page in pdf_reader.pages:
                text += page.extract_text()
            
            with open(txt_path, 'w', encoding='utf-8') as txt_file:
                txt_file.write(text)
            print(f"PDF '{pdf_path}' converted to TXT successfully!")
    except Exception as e:
        print(f"Error converting '{pdf_path}':", e)

def batch_convert_pdfs(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for pdf_file in os.listdir(input_folder):
        if pdf_file.endswith('.pdf'):
            pdf_path = os.path.join(input_folder, pdf_file)
            txt_filename = os.path.splitext(pdf_file)[0] + '.txt'
            txt_path = os.path.join(output_folder, txt_filename)
            convert_pdf_to_txt(pdf_path, txt_path)

In [7]:
# Replace 'input_folder' with the folder containing your PDF files
input_folder = r'C:\Users\achauha2\Downloads\policy_pdf'

# Replace 'output_folder' with the desired folder for the output TXT files
output_folder = r'C:\Users\achauha2\Downloads\policy_txt'

batch_convert_pdfs(input_folder, output_folder)

PDF 'C:\Users\achauha2\Downloads\policy_pdf\Leave and Holiday Policy .pdf' converted to TXT successfully!
PDF 'C:\Users\achauha2\Downloads\policy_pdf\Medical Insurance Policy.pdf' converted to TXT successfully!
PDF 'C:\Users\achauha2\Downloads\policy_pdf\Mobile Phone Reimbursement Policy.pdf' converted to TXT successfully!
PDF 'C:\Users\achauha2\Downloads\policy_pdf\Performance Appraisal.pdf' converted to TXT successfully!
PDF 'C:\Users\achauha2\Downloads\policy_pdf\PREVENTION OF SEXUAL HARASSMENT.pdf' converted to TXT successfully!
PDF 'C:\Users\achauha2\Downloads\policy_pdf\Promotion Policy.pdf' converted to TXT successfully!


In [8]:
acs = AzureSearch(azure_search_endpoint="https://ai-dev-ss-fts.search.windows.net",
                 azure_search_key="ICyDmM0S3sWaLpYwNGPfUOz7jblRyOHnE6p7uDHPRRAzSeAaLBFM",
                 index_name="hanu-policy-docs",
                 embedding_function= azure_embeddings.embed_query
                 )

In [11]:
loader = DirectoryLoader(r'C:\Users\achauha2\Downloads\test-txt', glob="*.txt", loader_cls=TextLoader, loader_kwargs={'autodetect_encoding': True})

documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# Add documents to Azure Search
acs.add_documents(documents=chunks)

['OTBhZjY3YjgtZjY3ZC00NGFiLTk1N2ItYzQ1OTYwNGU5YjQ2',
 'MDA3NjJlODctNmJhNC00ODE0LTg0MzMtMmI5Mzk1MTcxNjM2',
 'MmUxNjUwYzctMWRlNy00MmEwLTk0MGYtMDVjYTY1ODA3NWFi',
 'NDJlZWQ3YzgtZjZhOC00NDFlLWIyMjgtYTlhMDBkYjczOTVi',
 'OTQ0YWQwZWMtZTc0ZC00OTgxLTk1YjUtYzhjMWNmMTAzMTBl',
 'MzYxOTZjYTYtYjNmNi00MTQ5LThlMjQtMDg0YTlmZjQ0ZWE5',
 'MmI4NzE1OWQtZmEzOC00MzA2LTgyNWQtMTlhMjI5YjI0M2Mw',
 'MjA5ZDhjNDktYTM0YS00YzI4LTlhNzAtMTgzYTNjM2VlYmVm',
 'MzdkNTE4MmQtOWY2MS00YjRhLTk1MjYtZmVhODNiZjU5MTEz',
 'ZWZkMTExM2QtNzQxYS00ZGNlLWEwMGUtYTYzZmU1OTg5M2E5',
 'M2M1N2QwOTItOTY2Ni00NzQzLTk0NTAtYzFhOGMxNmMxZmJm',
 'ZWY3OWQ2NTEtNGNiNi00NGFkLWEzNmItY2U2MWJjZjM5MDM5',
 'ZTVhZWQzMTQtMzAxYS00MzEzLTg1MDMtNjY2N2QyZWY1Nzkz',
 'MGIyNTZlMTYtMGYyMC00ZTY5LThlMzctYjliZDliOTg1Mjhi',
 'NDY4MTFiNjgtYThhMS00Y2Y1LWEzNzctYzU2YzViZjk1MjEx',
 'M2I0ZTUwZWItODQ4NS00MWM2LTkzZTMtZjFhZWVkYTEwY2Y5',
 'NDEwMWJlMjktM2UwNi00ODFiLTgzMzItYWZiYWEyYjM4NDEw',
 'NmY1Mzk2ZDItMjVjOC00NzJjLTk1NGQtNDI2OWZiYTNiOGQ3',
 'ZTI0YjkzYjYtMDhlMS00N2Y4LTg5MWUtNGMzMTk4MjU3

In [12]:
retriever = acs.as_retriever(search_kwargs={"k": 3})

In [13]:
from langchain import PromptTemplate

template = """As an assistant, your role is to assist company employees with their 
policy plan inquiries and questions about the employee handbook. Your responses 
should be concise and based solely on the facts provided in the sources listed below.
If the information provided is insufficient, state that you don’t know. 
Avoid generating answers that do not utilize the sources below. If a clarifying 
question to the user would be helpful, feel free to ask.

Question: {question} 
Context: {context} 
Answer:
"""

prompt = PromptTemplate(
    template=template, 
    input_variables=["context","question"]
  )

print(prompt)

input_variables=['context', 'question'] template='As an assistant, your role is to assist company employees with their \npolicy plan inquiries and questions about the employee handbook. Your responses \nshould be concise and based solely on the facts provided in the sources listed below.\nIf the information provided is insufficient, state that you don’t know. \nAvoid generating answers that do not utilize the sources below. If a clarifying \nquestion to the user would be helpful, feel free to ask.\n\nQuestion: {question} \nContext: {context} \nAnswer:\n'


In [14]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | azure_model
    | StrOutputParser() 
)

In [15]:
# Ragas wants ['question', 'answer', 'contexts', 'ground_truths'] as
'''
{
    "question": [], <-- question from faq doc
    "answer": [], <-- answer from generated result
    "contexts": [], <-- context
    "ground_truths": [] <-- actual answer
}
'''

from datasets import Dataset

questions = ["How many Casual leaves can be taken in a year under Hanu Leave Policy ?", 
             "What is the Performance Appraisal Policy at Hanu ?",
             "What is the total reimbursement amount for both mobile+internet for rockstars at Hanu?",
            ]
ground_truths = [["10"],
                ["Hanu Software Solutions India Pvt. Ltd. has a Performance Appraisal Policy that defines the guidelines and due processes for Rockstars (employees) to undergo performance evaluation and appraisal. The Performance Appraisal Cycle spreads from January to December, and it consists of Rockstars' reviews of goals and a salary revision exercise. Reviews are done quarterly, and salary revision is taken once a year. The policy covers all Rockstars of Hanu Software Solutions India across different departments and Pan India geographies."],
                ["800 rupees."]]
answers = []
contexts = []

# Inference
for query in questions:
    answers.append(rag_chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [16]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 3
})

In [17]:
from ragas.metrics import (
    context_precision,
    answer_relevancy,
    faithfulness,
    context_recall,
)
from ragas.metrics.critique import harmfulness

# list of metrics we're going to use
metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    harmfulness,
]

In [18]:
# wrapper around azure_model
ragas_azure_model = LangchainLLM(azure_model)
# patch the new RagasLLM instance
answer_relevancy.llm = ragas_azure_model

In [19]:
# embeddings can be used as it is
answer_relevancy.embeddings = azure_embeddings

In [20]:
for m in metrics:
    m.__setattr__("llm", ragas_azure_model)

In [21]:
result = evaluate(
    dataset,
    metrics=metrics,
)

result

evaluating with [faithfulness]


100%|██████████| 1/1 [00:04<00:00,  4.79s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:07<00:00,  7.20s/it]


evaluating with [context_recall]


...

evaluating with [context_precision]


100%|██████████| 1/1 [00:01<00:00,  1.79s/it]


evaluating with [harmfulness]


100%|██████████| 1/1 [00:01<00:00,  1.35s/it]


{'faithfulness': 0.1667, 'answer_relevancy': 0.0000, 'context_recall': 0.0000, 'context_precision': 0.0000, 'harmfulness': 0.0000}

In [22]:
df = result.to_pandas()
df.head()

,question,answer,contexts,ground_truths,faithfulness,answer_relevancy,context_recall,context_precision,harmfulness
0,How many Casual leaves can be taken in a year ...,"I'm sorry, I cannot find any information about...","[/partner, and \nother c lose relatives so the...",[10],0.0,0.0,0.0,0.0,0
1,What is the Performance Appraisal Policy at Ha...,The Performance Appraisal Policy at Hanu is no...,[1 \n Information Security & Privacy Training ...,[Hanu Software Solutions India Pvt. Ltd. has a...,0.0,0.0,0.0,0.0,0
2,What is the total reimbursement amount for bot...,"I'm sorry, I cannot find any information about...",[1 \n Information Security & Privacy Training ...,[800 rupees.],0.5,0.0,0.0,0.0,0
